In [1]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
import joblib


In [2]:
def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError

In [3]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [16]:
'''
fetch RFH_without_GAC
'''

from __future__ import division
import sys
import numpy as np
import pandas as pd
def convert():
    """RFH feature"""
    dataPath= "/home02/chenhuangrong/Homo_sapiens_used_test.fasta"
    lines=open(dataPath).readlines()
    outputPath="/home02/chenhuangrong/GAC_RFH_homo.csv"
    classifier_type='U'#sys.argv[3]
    finally_text = open(outputPath, 'w')
    finnaly_lines=""
    for line in lines:
        if line.strip()=="":continue
        if line.strip()[0] in ['A','G','C',classifier_type]:
            position_mark=0
            count_AGCT=[0,0,0,0]
            temp = ""
            one_line=list(line.strip())
            for i,x in enumerate(one_line):
                if i!=24 and i!=25 and i!=26:
                    position_mark+=1
                    if x=="A" or x=="G":temp+="1,"
                    else:temp+="0,"
                    if x=="A" or x==classifier_type:temp+="1,"
                    else:temp+="0,"
                    if x == "A" or x == "C":temp+="1,"
                    else:temp+="0,"
                    if x == "A":
                        count_AGCT[0] += 1
                        temp +=str(round(count_AGCT[0] / position_mark*1.0,2))
                        temp+=','
                    elif x == "G":
                        count_AGCT[1] += 1
                        temp +=str(round(count_AGCT[1] / position_mark*1.0,2))
                        temp += ','
                    elif x == "C":
                        count_AGCT[2] += 1
                        temp +=str(round(count_AGCT[2] / position_mark*1.0,2))
                        temp += ','
                    elif x == classifier_type:
                        count_AGCT[3] += 1
                        temp +=str(round(count_AGCT[3] / position_mark*1.0,2))
                        temp += ','

            finnaly_lines+=((temp[:len(temp)-1])+'\n')
            #finally_text.write(temp+'\n')
    finally_text.writelines(finnaly_lines)
    finally_text.close()
convert()
data=pd.read_csv('/home02/chenhuangrong/GAC_RFH_homo.csv',header=None,index_col=False)
data=data.values
print data.shape

(16732, 192)


In [20]:
'''
fetch PCP feature
'''
from array import array
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm, model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
import sys
# out_file=open("phisical_chemical_parameter_and_label.txt",'w+')
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return SN,SP
seq=[]
physical_chemical_properties_path="/home02/chenhuangrong/physical_chemical_properties.txt"#sys.argv[1]
m6a_2614_sequence="/home02/chenhuangrong/Homo_sapiens_used_test.fasta"#sys.argv[2]

fh=open(m6a_2614_sequence)
for line in fh:#get the fasta sequence
    if line.startswith('>'):
#         print line.replace('\n','')
        pass
    else:
        seq.append(line.replace('\r\n',''))
fh.close()

data=pd.read_csv(physical_chemical_properties_path,header=None,index_col=None)#read the phisical chemichy proporties
prop_key=data.values[:,0]
prop_data=data.values[:,1:]
prop_data=np.matrix(prop_data)
DNC_value=np.array(prop_data).T
DNC_value_scale=[[] for e in xrange(len(DNC_value))]
for i in xrange(len(DNC_value)):
    average_=sum(DNC_value[i]*1.0/len(DNC_value[i]))
    std_=np.std(DNC_value[i],ddof=1)
    DNC_value_scale[i]=[round((e-average_)/std_,2) for e in DNC_value[i]]
prop_data_transformed=zip(*DNC_value_scale)

# prop_data_transformed=StandardScaler().fit_transform(prop_data)
prop_len=len(prop_data_transformed[0])
print prop_len
whole_m6a_seq=seq
i=0
phisical_chemichy_len=len(prop_data_transformed)#the length of properties
sequence_line_len=len(seq[0])#the length of one sequence
print seq[0]
print sequence_line_len
LAMDA=4
finally_result=[]#used to save the fanal result
for one_m6a_sequence_line in whole_m6a_seq:
    one_sequence_value=[[]]*(sequence_line_len-1)
    PC_m=[0.0]*prop_len
    PC_m=np.array(PC_m)
    for one_sequence_index in range(sequence_line_len-1):
        for prop_index in xrange(16):
            if one_m6a_sequence_line[one_sequence_index:one_sequence_index+2]==prop_key[prop_index]:
                one_sequence_value[one_sequence_index]=prop_data_transformed[prop_index]
                break
        PC_m+=np.array(one_sequence_value[one_sequence_index])
    PC_m=PC_m/(sequence_line_len-1)
    auto_value=[]
    for LAMDA_index in xrange(1,LAMDA+1):
        temp = [0.0] * prop_len
        temp=np.array(temp)
        for auto_index in xrange(1,sequence_line_len-LAMDA_index):
            temp=temp+(np.array(one_sequence_value[auto_index-1])-PC_m)*(np.array(one_sequence_value[auto_index+LAMDA_index-1])-PC_m)
            temp=[round(e,8) for e in temp.astype(float)]
        x=[round(e/(sequence_line_len-LAMDA_index-1),8) for e in temp]
        auto_value.extend([round(e,8) for e in x])
    for LAMDA_index in xrange(1, LAMDA + 1):
        for i in xrange(1,prop_len+1):
            for j in xrange(1,prop_len+1):
                temp2=0.0
                if i != j:
                    for auto_index in xrange(1, sequence_line_len - LAMDA_index):
                            temp2+=(one_sequence_value[auto_index-1][i-1]-PC_m[i-1])*(one_sequence_value[auto_index+LAMDA_index-1][j-1]-PC_m[j-1])
                    auto_value.append(round(temp2/((sequence_line_len-1)-LAMDA_index),8))

    finally_result.append(auto_value)
# finally_result=MinMaxScaler().fit_transform(np.matrix(finally_result))
print len(finally_result)
pd.DataFrame(finally_result).to_csv('/home02/chenhuangrong/PCP_homo.csv',header=None,index=False)



# out_file.close()


10
CAAAGGUGACCCACUUCGUUCAUGGACGUUCCCUGAAAUCAGGGACACUAU
51
2614


In [7]:
'''
fetch PSNP feature
'''


import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
# from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import copy
import random
import joblib
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0




def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN


# sequences_RFH_convert('mutation_test.txt')#get RFH feature and save into temp.csv
# sequences_RFH_convert('m6a_data.txt')#get RFH feature and save into temp.csv

seq=[]

m6a_2614_sequence="/home02/chenhuangrong/m6a_data.txt"
# m6a_2614_sequence="mutation_test.txt"
# m6a_2614_sequence="m6a_data.txt"


RNA_code='ACGU'
k=1
interval=1
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        pass
    else:
        seq.append(line.replace('\r\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
print len(seq)
positive_seq=seq[:int(len(seq)/2)]
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_data, train_target, test_size=0.1, random_state=0)
negative_seq=seq[int(len(seq)/2):]
kf = KFold(n_splits=division_num,shuffle=False)  

code_values=make_kmer_list(1,RNA_code)
code_len=len(code_values)
PSNP_data=[[0 for ii in xrange(len(seq[0])-interval)] for jj in xrange(len(seq))]
positive_seq_value=pd.read_csv('positive_seq_value_',index_col=False,header=None)
positive_seq_value=positive_seq_value.values
print len(positive_seq_value[0])
negative_seq_value=pd.read_csv('negative_seq_value_',index_col=False,header=None)
negative_seq_value=negative_seq_value.values
for i,line_value in enumerate(seq):
    for j,code_value in enumerate(line_value):
        if j<= len(line_value)-interval-1:
            for p,c_value in enumerate(code_values):
                if c_value==line_value[j:j+1]:
                    PSNP_data[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
print len(PSNP_data)
pd.DataFrame(PSNP_data).to_csv('/home02/chenhuangrong/PSNP_res.csv',header=None,index=False)

2614
51
2614


In [6]:
'''
this is PSNP
'''
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import joblib

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence="/home02/chenhuangrong/m6a_data.txt"
RNA_code='ACGU'
# k=4
interval=1
gap=0
divided_num=10.0
division_num=10
fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\r\n',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:int(len(seq)//2)]
print int(len(seq)//2)
# positive_seq=pd.DataFrame(positive_seq)
negative_seq=seq[int(len(seq)//2):]
# negative_seq=pd.DataFrame(negative_seq)
# for interval in xrange(1,k+1):
final_seq_value=[[0 for ii in xrange(len(seq[0]))] for jj in xrange(len(seq))]
code_values=make_kmer_list(interval,RNA_code)
code_len=len(code_values)
positive_seq_value=[[0 for jj in xrange(len(seq[0]))] for ii in xrange(code_len)]
negative_seq_value=[[0 for jj in xrange(len(seq[0]))] for ii in xrange(code_len)]
for i,line_value in enumerate(positive_seq):
    for j,code_value in enumerate(line_value):
        if j<= len(line_value)-1 :
            for p,c_value in enumerate(code_values):
                temp_value=np.array([y for x,y in enumerate(line_value[j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                temp_value=[reduce(lambda x,y:x+y,temp_value)]
                temp_value=temp_value[0] 
                if c_value==temp_value:
                    positive_seq_value[p][j]+=1
positive_seq_value=np.matrix(positive_seq_value)*1.0/(len(seq)//2)
pd.DataFrame(positive_seq_value).to_csv('positive_seq_value_',index=False,header=None)
for i,line_value in enumerate(negative_seq):
    for j,code_value in enumerate(line_value):
        if j<= len(line_value)-1:
            for p,c_value in enumerate(code_values):
                temp_value=np.array([y for x,y in enumerate(line_value[j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                temp_value=[reduce(lambda x,y:x+y,temp_value)]
                temp_value=temp_value[0]
                if c_value==temp_value:
                    negative_seq_value[p][j]+=1
negative_seq_value=np.matrix(negative_seq_value)*1.0/(len(seq)//2)
pd.DataFrame(negative_seq_value).to_csv('negative_seq_value_',index=False,header=None)
for i,line_value in enumerate(seq):
    for j,code_value in enumerate(line_value):
        if j<= len(line_value)-1:
            for p,c_value in enumerate(code_values):
                temp_value=np.array([y for x,y in enumerate(line_value[j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
                temp_value=[reduce(lambda x,y:x+y,temp_value)]
                temp_value=temp_value[0]
                if c_value==temp_value:
                      final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]
# y_final_seq_value=[[0 for ii in xrange(len(seq[0])-interval-gap*(interval-1))] for jj in xrange(len(positive_negative_y_train))]
# for i,line_value in enumerate(positive_negative_y_train.values):
#     for j,code_value in enumerate(line_value[0]):
#         if j<= len(line_value[0])-interval-1-gap*(interval-1) :
#             for p,c_value in enumerate(code_values):
#                 temp_value=np.array([y for x,y in enumerate(line_value[0][j:j+interval+gap*(interval-1)]) if x%(gap+1)==0])
#                 temp_value=[reduce(lambda x,y:x+y,temp_value)]
#                 temp_value=temp_value[0]
#                 if c_value==temp_value:
#                       y_final_seq_value[i][j]=positive_seq_value[p,j]-negative_seq_value[p,j]







#     print len(y_final_seq_value),len(y_final_seq_value[0]),y_final_seq_value
#     break
X_train = np.array(final_seq_value)
pd.DataFrame(X_train).to_csv('/home02/chenhuangrong/PSNP_sac.csv',header=None,index=False)
Y_train = list(map(lambda x: 1, xrange(len(final_seq_value) // 2)))
Y2_train = list(map(lambda x: 0, xrange(len(final_seq_value) // 2)))
Y_train.extend(Y2_train)
Y_train = np.array(Y_train)
print Y_train
# X_test = np.array(y_final_seq_value)
# Y_test  = list(map(lambda x: 1, xrange(len(y_final_seq_value) / 2)))
# Y2_test  = list(map(lambda x: 0, xrange(len(y_final_seq_value) / 2)))
# Y_test.extend(Y2_test )
# Y_test  = np.array(Y_test)

svc = svm.SVC(probability=True)
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
# parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X_train, Y_train)
joblib.dump(clf,'/home02/chenhuangrong/model/PSNP.model')


    
        



    

1307
[1 1 1 ..., 0 0 0]


['/home02/chenhuangrong/model/PSNP.model']

In [3]:
import joblib
joblib.dump(clf,'/home02/chenhuangrong/model/PSDNP_1gap.model')

['/home02/chenhuangrong/model/PSDNP_1gap.model']

In [45]:
'''
this is GAC_RFH
'''

import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence="/home02/chenhuangrong/GAC_RFH.csv"
RNA_code='ACGU'
k=1
interval=1
divided_num=10.0
division_num=10
seq=pd.read_csv(m6a_2614_sequence,header=None,index_col=None)
seq=seq.values
positive_seq=seq[0:1307,:]
# positive_seq=shuffle(positive_seq,random_state=0)
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_data, train_target, test_size=0.1, random_state=0)
negative_seq=seq[1307:2614,:]
# negative_seq=shuffle(negative_seq,random_state=0)
# kf = KFold(n_splits=division_num,shuffle=False)  
# for train_index , test_index in kf.split(positive_seq):  
#     positive_df=pd.DataFrame(positive_seq)
#     positive_x_train=positive_df.iloc[train_index,:]
#     positive_y_train=positive_df.iloc[test_index,:]
#     negative_df=pd.DataFrame(negative_seq)
#     negative_x_train=negative_df.iloc[train_index,:]
#     negative_y_train=negative_df.iloc[test_index,:]
#     positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
#     positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
    
X_train = np.array(seq)
Y_train = list(map(lambda x: 1, xrange(len(positive_seq) )))
Y2_train = list(map(lambda x: 0, xrange(len(negative_seq) )))
Y_train.extend(Y2_train)
Y_train = np.array(Y_train)
print Y_train
print len(Y_train)
#     X_test = np.array(positive_negative_y_train)
#     Y_test  = list(map(lambda x: 1, xrange(len(positive_negative_y_train) / 2)))
#     Y2_test  = list(map(lambda x: 0, xrange(len(positive_negative_y_train) / 2)))
#     Y_test.extend(Y2_test )
#     Y_test  = np.array(Y_test)
#     print "train and test samples finished"
svc = svm.SVC(probability=True)
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
print "begin grid search"


clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X_train, Y_train)
# joblib.dump(clf,'/home02/chenhuangrong/model/GAC_RFH.model')
C=clf.best_params_['C']

gamma=clf.best_params_['gamma']

cl=svm.SVC(probability=True,kernel='rbf',C=C,gamma=gamma)
cl.fit(X_train, Y_train)
joblib.dump(cl,'/home02/chenhuangrong/model/GAC_RFH_cl.model')

#     C=clf.best_params_['C']
#     y_pred_prob=clf.predict_proba(X_test)
#     gamma=clf.best_params_['gamma']
#     print 'c:',C,'gamma:',gamma
#     y_pred=clf.predict(X_test)
    
#     y_pred_prob_all.extend(y_pred_prob)
#     y_pred_all.extend(y_pred)
#     Y_all.extend(Y_test)
# #     ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
#     ACC=metrics.accuracy_score(Y_test,y_pred)
#     print ACC
#     precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
#     F1_Score=metrics.f1_score(Y_test, y_pred)
#     F_measure=F1_Score
#     MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
#     pos=TP+FN
#     neg=FP+TN
#     ACC_all=ACC_all+ACC
#     print "ACC_all:",ACC_all
#     precision_all=precision_all+precision
#     recall_all=recall_all+recall
#     SN_all=SN_all+SN
#     SP_all=SP_all+SP
#     GM_all=GM_all+GM
#     TP_all=TP_all+TP
#     TN_all=TN_all+TN
#     FP_all=FP_all+FP
#     FN_all=FN_all+FN
#     F_measure_all=F_measure_all+F_measure
#     F1_Score_all=F1_Score_all+F1_Score
#     pos_all=pos_all+pos
#     neg_all=neg_all+neg
#     MCC_all=MCC_all+MCC
# all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
# pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/GAC_RFH_independent_predict.csv',header=None,index=False)
# fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
# roc_auc = auc(fpr, tpr)
# savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
#             SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
#             FN_all,FP_all,TN_all,pos_all,neg_all]]]
# print savedata
# easy_excel.save("tenfold_crossvalidation",[str(X_train.shape[1])],savedata,'/home02/chenhuangrong/GAC_RFH_independent_.xls')

[1 1 1 ..., 0 0 0]
2614
begin grid search


['/home02/chenhuangrong/model/GAC_RFH_cl.model']

In [ ]:
'''
this is PCP
'''

final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0) #above was used to generate xlsx format Excel file
RNA_code='ACGU' 
interval=3 # RNA_code and interval used mix used to generate AAA AAC AAG ...
division_num=10
divided_num=np.float(division_num)# ten fold so the result should divided by ten


# seq=[e.replace('U','A') for e in seq]
# seq=[e.replace('G','C') for e in seq]
seq=pd.read_csv('/home02/chenhuangrong/PCP.csv',header=None,index_col=False)
seq=seq.values

positive_seq=seq[:len(seq)//2]
positive_seq=pd.DataFrame(positive_seq)
negative_seq=seq[len(seq)//2:]
negative_seq=pd.DataFrame(negative_seq)

seq=pd.DataFrame(seq)

#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end




#generate train and test data begining
X_train = np.array(seq.values)
Y_train = list(map(lambda x: 1, xrange(len(X_train) // 2)))
Y2_train = list(map(lambda x: 0, xrange(len(X_train) // 2)))
Y_train.extend(Y2_train)
Y_train = np.array(Y_train)



# training model and optimized parameters of C and gamma begining
svc = svm.SVC(probability=True)
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,28)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,28))}
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X_train, Y_train)
joblib.dump(clf,'/home02/chenhuangrong/model/PCP.model')
#     # training model and optimized parameters of C and gamma end
    
    
#     #print best C and gamma begining
#     C=clf.best_params_['C']
#     gamma=clf.best_params_['gamma']
#     print 'c:',C,'gamma:',gamma
#     #print best C and gamma end
    
    
#     #getting predict probability and predict label begining
#     y_pred_prob=clf.predict_proba(X_test)
#     y_pred=clf.predict(X_test)
#     #getting predict probability and predict label begining

#     #the process of generating  usefule data begining
#     y_pred_prob_all.extend(y_pred_prob)
#     y_pred_all.extend(y_pred)
#     Y_all.extend(Y_test)
#     ACC=metrics.accuracy_score(Y_test,y_pred)
#     print ACC
#     precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
#     F1_Score=metrics.f1_score(Y_test, y_pred)
#     F_measure=F1_Score
#     MCC=metrics.matthews_corrcoef(Y_test, y_pred)

#     pos=TP+FN
#     neg=FP+TN
#     ACC_all=ACC_all+ACC
#     print "ACC_all:",ACC_all
#     precision_all=precision_all+precision
#     recall_all=recall_all+recall
#     SN_all=SN_all+SN
#     SP_all=SP_all+SP
#     GM_all=GM_all+GM
#     TP_all=TP_all+TP
#     TN_all=TN_all+TN
#     FP_all=FP_all+FP
#     FN_all=FN_all+FN
#     F_measure_all=F_measure_all+F_measure
#     F1_Score_all=F1_Score_all+F1_Score
#     pos_all=pos_all+pos
#     neg_all=neg_all+neg
#     MCC_all=MCC_all+MCC
#     #the process of generating  usefule data end
    
# #the process of save  data to disk begining   
# all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
# pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PCP_independent_predict.csv',header=None,index=False)
# fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
# roc_auc = auc(fpr, tpr)
# savedata=[str(X_train.shape[1]),"正："+str(pos_all)+'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
#             SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
#             FN_all,FP_all,TN_all]
# final_out_to_excel.append(savedata)

# print savedata

# pd.DataFrame(final_out_to_excel).to_excel("/home02/chenhuangrong/PCP_independent.xlsx",sheet_name="independent_test",index=False,header=False)
#the process of save  data to disk end

In [ ]:
#!/usr/bin/env python
# encoding:utf-8
# print(__doc__)

'''
drawing ROC curves
'''

import numpy as np
import pandas as pd
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm
from sklearn.metrics import roc_curve, auc,precision_recall_curve

plt.figure(figsize=(8,6), dpi=80, facecolor=(1, 1, 1))
data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_1.csv",header=None,index_col=None)
fpr, tpr, thresholds = roc_curve(data.values[:,0], data.values[:, 2])
roc_auc = auc(fpr, tpr)
# plt.plot(fpr, tpr,'r-',label=u'PS(1-mer)NP(ROC =%f)'%roc_auc)
plt.plot(fpr, tpr,'r-',label=u'PS(1-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_2.csv",header=None,index_col=None)
fpr, tpr, thresholds = roc_curve(data.values[:,0], data.values[:, 2])
roc_auc = auc(fpr, tpr)
# plt.plot(fpr, tpr,'b-.',label=u'PS(2-mer)NP(ROC =%f)'%roc_auc)
plt.plot(fpr, tpr,'b-.',label=u'PS(2-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_3.csv",header=None,index_col=None)
fpr, tpr, thresholds = roc_curve(data.values[:,0], data.values[:, 2])
roc_auc = auc(fpr, tpr)
# plt.plot(fpr, tpr,'k,',label=u'PS(3-mer)NP(ROC =%f)'%roc_auc)
plt.plot(fpr, tpr,'k,',label=u'PS(3-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_4.csv",header=None,index_col=None)
fpr, tpr, thresholds = roc_curve(data.values[:,0], data.values[:, 2])
roc_auc = auc(fpr, tpr)
# plt.plot(fpr, tpr,linestyle='--', lw=2, color='r',alpha=0.3,label='svm(ROC =%f)'%roc_auc)
# plt.plot(fpr, tpr,linestyle='g-', lw=1,alpha=0.3,label='svm(ROC =%f)'%roc_auc)
# plt.plot(fpr, tpr,'g--',label=u'PS(4-mer)NP(ROC =%f)'%roc_auc)
plt.plot(fpr, tpr,'g--',label=u'PS(4-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_5.csv",header=None,index_col=None)
fpr, tpr, thresholds = roc_curve(data.values[:,0], data.values[:, 2])
roc_auc = auc(fpr, tpr)
# plt.plot(fpr, tpr,linestyle='--', lw=2, color='r',alpha=0.3,label='svm(ROC =%f)'%roc_auc)
# plt.plot(fpr, tpr,linestyle='g-', lw=1,alpha=0.3,label='svm(ROC =%f)'%roc_auc)
# plt.plot(fpr, tpr,'g-',label=u'PS(5-mer)NP(ROC =%f)'%roc_auc)
plt.plot(fpr, tpr,'y-,',label=u'PS(5-mer)NP')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
# plt.savefig('F:/roc.tif',dpi=300)
plt.show()

In [ ]:
#!/usr/bin/env python
# encoding:utf-8
# print(__doc__)


'''

drawing PRC curves
'''
import numpy as np
import pandas as pd
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm
from sklearn.metrics import roc_curve, auc,precision_recall_curve

plt.figure(figsize=(8,6), dpi=80, facecolor=(1, 1, 1))
data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_1.csv",header=None,index_col=None)
fpr, tpr, thresholds = precision_recall_curve(data.values[:,0], data.values[:, 2])
plt.plot(tpr, fpr,'r-',label=u'PS(1-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_2.csv",header=None,index_col=None)
fpr, tpr, thresholds = precision_recall_curve(data.values[:,0], data.values[:, 2])
plt.plot(tpr, fpr,'b-.',label=u'PS(2-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_3.csv",header=None,index_col=None)
fpr, tpr, thresholds = precision_recall_curve(data.values[:,0], data.values[:, 2])
plt.plot(tpr, fpr,'k,',label=u'PS(3-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_4.csv",header=None,index_col=None)
fpr, tpr, thresholds = precision_recall_curve(data.values[:,0], data.values[:, 2])
plt.plot(tpr, fpr,'g--',label=u'PS(4-mer)NP')

data=pd.read_csv(u"F:/杂货铺/PSTNP_results_independent_test/independent_test_5.csv",header=None,index_col=None)
fpr, tpr, thresholds = precision_recall_curve(data.values[:,0], data.values[:, 2])
plt.plot(tpr, fpr,'y-,',label=u'PS(5-mer)NP')

plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()
# plt.savefig('F:/roc.tif',dpi=300)

In [38]:
'''
PCP load joblib
'''
import joblib
divided_num=1.0
#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end



clf=joblib.load('/home02/chenhuangrong/model/PCP.model')
# PCP_data=pd.read_csv('/home02/chenhuangrong/PCP_homo.csv',header=None,index_col=False)
PCP_data=pd.read_csv('/home02/chenhuangrong/PCP_homo.csv',header=None,index_col=False)
PCP_data=PCP_data.values
y_true=[1 if index<len(PCP_data)//2 else 0 for index,e in enumerate(PCP_data)]


C=clf.best_params_['C']
y_pred_prob=clf.predict_proba(PCP_data)

gamma=clf.best_params_['gamma']
print 'c:',C,'gamma:',gamma
y_pred=clf.predict(PCP_data)

y_pred_prob_all.extend(y_pred_prob)
y_pred_all.extend(y_pred)
Y_all.extend(y_true)
ACC=metrics.accuracy_score(y_true,y_pred)
print ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(y_true, y_pred) 
F1_Score=metrics.f1_score(y_true, y_pred)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(y_true, y_pred)

pos=TP+FN
neg=FP+TN
ACC_all=ACC_all+ACC
print "ACC_all:",ACC_all
precision_all=precision_all+precision
recall_all=recall_all+recall
SN_all=SN_all+SN
SP_all=SP_all+SP
GM_all=GM_all+GM
TP_all=TP_all+TP
TN_all=TN_all+TN
FP_all=FP_all+FP
FN_all=FN_all+FN
F_measure_all=F_measure_all+F_measure
F1_Score_all=F1_Score_all+F1_Score
pos_all=pos_all+pos
neg_all=neg_all+neg
MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PCP_idepe_homo.csv',header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
# easy_excel.save("svm_independent_test",[str(X_train.shape[1])],savedata,'/home02/chenhuangrong/PCP_idepe_homo.xls')
easy_excel.save("svm_independent_test",[str(PCP_data.shape[1])],savedata,'/home02/chenhuangrong/PCP_idepe_homo.xls')




# y_pred=clf.predict(y_true)
# y_pred_prob=clf.predict_prob(y_true)
# precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(y_true, y_pred)
# F1_Score=metrics.f1_score(y_true, y_pred)
# F_measure=F1_Score
# MCC=metrics.matthews_corrcoef(y_true, y_pred)
# pos=TP+FN
# neg=FP+TN
# fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
# roc_auc = auc(fpr, tpr)
# savedata=[[['svm',ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,roc_auc,TP,FN,FP,TN,pos,neg]]]
# print savedata
# easy_excel.save("svm_crossvalidation",["PCP_homo"],savedata,'/home02/chenhuangrong/PCP_Homo.xls')
    
           


c: 5.30520098287 gamma: 0.03125
0.569328233325
ACC_all: 0.569328233325
[[['svmC:5.30520098287gamma:0.03125', 0.56932823332536453, 0.6207327227310575, 0.3564427444417882, 0.3564427444417882, 0.7822137222089409, 0.5280287926658749, 0.45284738041002282, 0.45284738041002282, 0.1532401874782022, 0.60533886462223241, 2982.0, 5384.0, 1822.0, 6544.0, 8366.0, 8366.0]]]


True

In [44]:
# !/use/bin/env python
# encoding:utf-8
'''
PSDNP_1gap load joblib
'''
import joblib
from sklearn.metrics import roc_curve, auc
divided_num=1.0
#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end



clf=joblib.load('/home02/chenhuangrong/model/PSDNP_1gap.model')#PSDNP_1gap.model
# PCP_data=pd.read_csv('/home02/chenhuangrong/PSDNP_1gap_homo_res.csv',header=None,index_col=False)
PCP_data=pd.read_csv('/home02/chenhuangrong/PSDNP_1gap_sac.csv',header=None,index_col=False)
PCP_data=PCP_data.values
print PCP_data.shape
y_true=[1 if index<PCP_data.shape[0]//2 else 0 for index,e in enumerate(PCP_data)]
print len(y_true)
print y_true

C=clf.best_params_['C']
y_pred_prob=clf.predict_proba(PCP_data)

gamma=clf.best_params_['gamma']
print 'c:',C,'gamma:',gamma
y_pred=clf.predict(PCP_data)

y_pred_prob_all.extend(y_pred_prob)
y_pred_all.extend(y_pred)
Y_all.extend(y_true)
ACC=metrics.accuracy_score(y_true,y_pred)
print ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(y_true, y_pred) 
F1_Score=metrics.f1_score(y_true, y_pred)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(y_true, y_pred)

pos=TP+FN
neg=FP+TN
ACC_all=ACC_all+ACC
print "ACC_all:",ACC_all
precision_all=precision_all+precision
recall_all=recall_all+recall
SN_all=SN_all+SN
SP_all=SP_all+SP
GM_all=GM_all+GM
TP_all=TP_all+TP
TN_all=TN_all+TN
FP_all=FP_all+FP
FN_all=FN_all+FN
F_measure_all=F_measure_all+F_measure
F1_Score_all=F1_Score_all+F1_Score
pos_all=pos_all+pos
neg_all=neg_all+neg
MCC_all=MCC_all+MCC

all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]



pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PSDNP_1gap_idepe_self.csv',header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save("svm_independent_test",[str(PCP_data.shape[1])],savedata,'/home02/chenhuangrong/PSDNP_1gap_idepe_self.xls')



# y_pred=clf.predict(y_true)
# y_pred_prob=clf.predict_prob(y_true)
# precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(y_true, y_pred)
# F1_Score=metrics.f1_score(y_true, y_pred)
# F_measure=F1_Score
# MCC=metrics.matthews_corrcoef(y_true, y_pred)
# pos=TP+FN
# neg=FP+TN
# fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
# roc_auc = auc(fpr, tpr)
# savedata=[[['svm',ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,roc_auc,TP,FN,FP,TN,pos,neg]]]
# print savedata
# easy_excel.save("svm_crossvalidation",["PCP_homo"],savedata,'/home02/chenhuangrong/PCP_Homo.xls')
    
           


(2614, 49)
2614
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

True

In [47]:
'''
GAC_RFH_ load joblib
'''

import joblib
divided_num=1.0
#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end


clf=joblib.load('/home02/chenhuangrong/model/GAC_RFH_cl.model')
PCP_data=pd.read_csv('/home02/chenhuangrong/GAC_RFH_homo.csv',header=None,index_col=False)
PCP_data=PCP_data.values
y_true=[1 if index<len(PCP_data)//2 else 0 for index,e in enumerate(PCP_data)]


# C=clf.best_params_['C']
y_pred_prob=clf.predict_proba(PCP_data)

# gamma=clf.best_params_['gamma']
# print 'c:',C,'gamma:',gamma
y_pred=clf.predict(PCP_data)

y_pred_prob_all.extend(y_pred_prob)
y_pred_all.extend(y_pred)
Y_all.extend(y_true)
ACC=metrics.accuracy_score(y_true,y_pred)
print ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(y_true, y_pred) 
F1_Score=metrics.f1_score(y_true, y_pred)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(y_true, y_pred)

pos=TP+FN
neg=FP+TN
ACC_all=ACC_all+ACC
print "ACC_all:",ACC_all
precision_all=precision_all+precision
recall_all=recall_all+recall
SN_all=SN_all+SN
SP_all=SP_all+SP
GM_all=GM_all+GM
TP_all=TP_all+TP
TN_all=TN_all+TN
FP_all=FP_all+FP
FN_all=FN_all+FN
F_measure_all=F_measure_all+F_measure
F1_Score_all=F1_Score_all+F1_Score
pos_all=pos_all+pos
neg_all=neg_all+neg
MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
# pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/GAC_RFH__idepe_homo.csv',header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
# easy_excel.save("svm_independent_test",[str(PCP_data.shape[1])],savedata,'/home02/chenhuangrong/GAC_RFH__idepe_homo.xls')



# y_pred=clf.predict(y_true)
# y_pred_prob=clf.predict_prob(y_true)
# precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(y_true, y_pred)
# F1_Score=metrics.f1_score(y_true, y_pred)
# F_measure=F1_Score
# MCC=metrics.matthews_corrcoef(y_true, y_pred)
# pos=TP+FN
# neg=FP+TN
# fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
# roc_auc = auc(fpr, tpr)
# savedata=[[['svm',ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,roc_auc,TP,FN,FP,TN,pos,neg]]]
# print savedata
# easy_excel.save("svm_crossvalidation",["PCP_homo"],savedata,'/home02/chenhuangrong/PCP_Homo.xls')
    
           


0.584927085824
ACC_all: 0.584927085824
[[['svmC:0.513002242354gamma:0.0374020008171', 0.58492708582357156, 0.6512667660208644, 0.36564666507291416, 0.36564666507291416, 0.804207506574229, 0.5422691147442112, 0.46834570925514807, 0.46834570925514807, 0.18899954146718589, 0.64882398268688735, 3059.0, 5307.0, 1638.0, 6728.0, 8366.0, 8366.0]]]
